In [1]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple, Union
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import random

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.common import (
    FitRes, EvaluateRes, Parameters, Scalar,
    parameters_to_ndarrays, ndarrays_to_parameters,
    Context
)
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

In [3]:
DEVICE = torch.device("cpu")  # Use "cuda" if available
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

Training on cpu
Flower 1.18.0 / PyTorch 2.6.0+cu124


In [4]:
# Hierarchical FL Configuration
NUM_EDGE_SERVERS = 2      # Number of edge servers
CLIENTS_PER_EDGE = 2      # Clients per edge server
TOTAL_CLIENTS = NUM_EDGE_SERVERS * CLIENTS_PER_EDGE
BATCH_SIZE = 32
EDGE_SERVER_ROUNDS = 2    # Local rounds at edge server before global aggregation

In [5]:
class Net(nn.Module):
    """Simple CNN for CIFAR-10"""
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
def get_parameters(net) -> List[np.ndarray]:
    """Extract model parameters as numpy arrays"""
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    """Set model parameters from numpy arrays"""
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [15]:
#  current load_datasets function uses IID partitions of CIFAR-10 training data via the IidPartitioner
def load_datasets(partition_id: int, num_partitions: int):
    """Load and partition CIFAR-10 dataset"""
    fds = FederatedDataset(dataset="cifar10", partitioners={"train": num_partitions})
    partition = fds.load_partition(partition_id)
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)

    pytorch_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    def apply_transforms(batch):
        batch["img"] = [pytorch_transforms(img) for img in batch["img"]]
        return batch

    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

In [16]:
def train(net, trainloader, epochs: int):
    """Train the network on the training set"""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"    Epoch {epoch+1}: loss {epoch_loss:.4f}, accuracy {epoch_acc:.4f}")

In [9]:
def test(net, testloader):
    """Evaluate the network on the test set"""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"], batch["label"]
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [10]:
class EdgeClient(NumPyClient):
    """
    Client that connects to an edge server
    """
    def __init__(self, client_id: int, edge_id: int, net, trainloader, valloader):
        self.client_id = client_id
        self.edge_id = edge_id
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"  [Client {self.client_id}@Edge{self.edge_id}] Sending parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        server_round = config.get("server_round", 1)
        local_epochs = config.get("local_epochs", 1)
        edge_round = config.get("edge_round", 1)

        print(f"  [Client {self.client_id}@Edge{self.edge_id}] Round {server_round}.{edge_round} - Training {local_epochs} epochs")

        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)

        return get_parameters(self.net), len(self.trainloader.dataset), {
            "edge_id": self.edge_id,
            "client_id": self.client_id
        }

    def evaluate(self, parameters, config):
        print(f"  [Client {self.client_id}@Edge{self.edge_id}] Evaluating")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

In [11]:
class EdgeServer(NumPyClient):
    """
    Edge Server that aggregates local clients and communicates with global server
    """
    def __init__(self, edge_id: int, client_apps: List[EdgeClient]):
        self.edge_id = edge_id
        self.client_apps = client_apps
        self.net = Net().to(DEVICE)
        self.edge_model_params = None

    def get_parameters(self, config):
        print(f"[EdgeServer {self.edge_id}] Sending aggregated model to Global Server")
        if self.edge_model_params is None:
            return get_parameters(self.net)
        return self.edge_model_params

    def fit(self, parameters, config):
        """
        Edge server aggregates its local clients multiple rounds before sending to global server
        """
        server_round = config.get("server_round", 1)
        edge_rounds = config.get("edge_rounds", EDGE_SERVER_ROUNDS)
        local_epochs = config.get("local_epochs", 1)

        print(f"[EdgeServer {self.edge_id}] Global Round {server_round} - Running {edge_rounds} edge rounds")

        # Initialize with global model
        set_parameters(self.net, parameters)
        current_params = get_parameters(self.net)

        # Run multiple edge aggregation rounds
        for edge_round in range(1, edge_rounds + 1):
            print(f"[EdgeServer {self.edge_id}] Edge Round {edge_round}/{edge_rounds}")

            # Simulate client training
            client_results = []
            client_weights = []

            # Each client trains with current edge model
            for client in self.client_apps:
                edge_config = {
                    "server_round": server_round,
                    "edge_round": edge_round,
                    "local_epochs": local_epochs
                }

                # Client training
                client_params, num_examples, metrics = client.fit(current_params, edge_config)
                client_results.append(client_params)
                client_weights.append(num_examples)

            # Aggregate client models (FedAvg)
            if client_results:
                current_params = self._federated_average(client_results, client_weights)

            print(f"[EdgeServer {self.edge_id}] Completed edge round {edge_round}")

        # Store final edge model
        self.edge_model_params = current_params
        total_examples = sum(client_weights) if client_weights else 1

        print(f"[EdgeServer {self.edge_id}] Completed all edge rounds, sending to global server")

        return self.edge_model_params, total_examples, {
            "edge_id": self.edge_id,
            "num_clients": len(self.client_apps),
            "edge_rounds_completed": edge_rounds,
            "total_client_examples": total_examples
        }

    def evaluate(self, parameters, config):
        """Evaluate edge server model"""
        print(f"[EdgeServer {self.edge_id}] Evaluating edge model")
        set_parameters(self.net, parameters)

        # Evaluate on a sample of client validation data
        total_loss, total_accuracy, total_samples = 0.0, 0.0, 0

        for client in self.client_apps:
            loss, accuracy = test(self.net, client.valloader)
            samples = len(client.valloader.dataset)
            total_loss += loss * samples
            total_accuracy += accuracy * samples
            total_samples += samples

        avg_loss = total_loss / total_samples if total_samples > 0 else 0.0
        avg_accuracy = total_accuracy / total_samples if total_samples > 0 else 0.0

        return float(avg_loss), total_samples, {"accuracy": float(avg_accuracy)}

    def _federated_average(self, client_params_list, weights):
        """Perform federated averaging of client parameters"""
        total_weight = sum(weights)

        if total_weight == 0:
            return client_params_list[0] if client_params_list else get_parameters(self.net)

        # Normalize weights
        weights = [w / total_weight for w in weights]

        # Initialize averaged parameters
        avg_params = []

        for layer_idx in range(len(client_params_list[0])):
            # Weighted average for this layer
            layer_avg = sum(
                weight * client_params[layer_idx]
                for client_params, weight in zip(client_params_list, weights)
            )
            avg_params.append(layer_avg)

        return avg_params

In [12]:
class HierarchicalEdgeStrategy(FedAvg):
    """
    Custom strategy for hierarchical FL with edge servers
    """
    def __init__(self, num_edge_servers: int, clients_per_edge: int, **kwargs):
        super().__init__(**kwargs)
        self.num_edge_servers = num_edge_servers
        self.clients_per_edge = clients_per_edge

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[Client, FitRes]],
        failures: List[Union[Tuple[Client, FitRes], Tuple[Client, Exception]]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Global aggregation of edge server models"""

        if not results:
            return None, {}

        print(f"\n🌐 [Global Server] Round {server_round}: Aggregating {len(results)} edge servers")

        # Extract edge server results
        edge_weights = []
        edge_params = []
        participating_edges = []

        total_examples = 0
        for client, fit_res in results:
            edge_id = fit_res.metrics.get("edge_id", 0)
            num_clients = fit_res.metrics.get("num_clients", 1)
            edge_rounds = fit_res.metrics.get("edge_rounds_completed", 1)

            participating_edges.append(edge_id)
            edge_weights.append(fit_res.num_examples)
            edge_params.append(parameters_to_ndarrays(fit_res.parameters))
            total_examples += fit_res.num_examples

            print(f"🏢 Edge Server {edge_id}: {fit_res.num_examples} examples from {num_clients} clients ({edge_rounds} edge rounds)")

        if not edge_params:
            return None, {}

        # Global federated averaging of edge server models
        total_weight = sum(edge_weights)
        weights = [w / total_weight for w in edge_weights]

        global_params = []
        for layer_idx in range(len(edge_params[0])):
            layer_avg = sum(
                weight * edge_params[edge_idx][layer_idx]
                for edge_idx, weight in enumerate(weights)
            )
            global_params.append(layer_avg)

        print(f"🌐 [Global Server] Aggregated {len(participating_edges)} edge servers")
        print(f"📊 Total examples: {total_examples}")

        return ndarrays_to_parameters(global_params), {
            "participating_edge_servers": len(participating_edges),
            "total_examples": total_examples,
            "edge_server_ids": participating_edges,
        }

In [13]:
def client_fn(context: Context) -> Client:
    """Factory function to create edge servers (not individual clients)"""
    node_id = context.node_id
    partition_id = context.node_config["partition-id"]

    # Each partition represents an edge server
    edge_id = partition_id
    print(f"\n🏢 Creating Edge Server {edge_id}")

    # Create clients for this edge server
    start_client = edge_id * CLIENTS_PER_EDGE
    end_client = start_client + CLIENTS_PER_EDGE

    edge_clients = []
    for client_id in range(start_client, end_client):
        print(f"  📱 Creating Client {client_id} for Edge Server {edge_id}")
        net = Net().to(DEVICE)
        trainloader, valloader, _ = load_datasets(client_id, TOTAL_CLIENTS)
        client = EdgeClient(client_id, edge_id, net, trainloader, valloader)
        edge_clients.append(client)

    edge_server = EdgeServer(edge_id, edge_clients)
    return edge_server.to_client()

def fit_config(server_round: int):
    """Configuration for each training round"""
    config = {
        "server_round": server_round,
        "local_epochs": 2,
        "edge_rounds": EDGE_SERVER_ROUNDS,
    }
    return config

def evaluate_fn(server_round: int, parameters, config):
    """Global evaluation function"""
    net = Net().to(DEVICE)
    _, _, testloader = load_datasets(0, TOTAL_CLIENTS)
    set_parameters(net, parameters)
    loss, accuracy = test(net, testloader)

    print(f"🌐 [Global Server] Round {server_round} - Global Test: loss {loss:.4f}, accuracy {accuracy:.4f}")
    return loss, {"accuracy": accuracy}

def server_fn(context: Context) -> ServerAppComponents:
    """Create the global server"""
    print(f"\n🌐 === Global Server Initialization ===")

    # Initialize global model
    net = Net().to(DEVICE)
    initial_params = get_parameters(net)

    # Create hierarchical strategy
    strategy = HierarchicalEdgeStrategy(
        num_edge_servers=NUM_EDGE_SERVERS,
        clients_per_edge=CLIENTS_PER_EDGE,
        fraction_fit=1.0,  # Use all edge servers
        fraction_evaluate=1.0,
        min_fit_clients=NUM_EDGE_SERVERS,
        min_evaluate_clients=NUM_EDGE_SERVERS,
        min_available_clients=NUM_EDGE_SERVERS,
        initial_parameters=ndarrays_to_parameters(initial_params),
        evaluate_fn=evaluate_fn,
        on_fit_config_fn=fit_config,
    )

    config = ServerConfig(num_rounds=3)
    return ServerAppComponents(strategy=strategy, config=config)

# Create apps
client_app = ClientApp(client_fn=client_fn)
server_app = ServerApp(server_fn=server_fn)

# Configure simulation
backend_config = {"client_resources": None}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_gpus": 1}}

print(f"\n🏗️ === Hierarchical FL Architecture ===")
print(f"🏢 Edge Servers: {NUM_EDGE_SERVERS}")
print(f"📱 Clients per Edge: {CLIENTS_PER_EDGE}")
print(f"📊 Total Clients: {TOTAL_CLIENTS}")
print(f"🔄 Edge Rounds per Global Round: {EDGE_SERVER_ROUNDS}")
print(f"\n🎯 Architecture Flow:")
print(f"   Global Server ↔ {NUM_EDGE_SERVERS} Edge Servers")
print(f"   Each Edge Server ↔ {CLIENTS_PER_EDGE} Clients")
print(f"   Edge Servers run {EDGE_SERVER_ROUNDS} local rounds before global aggregation")
print(f"\n🚀 Starting Client-Edge-Global FL Simulation...")




🏗️ === Hierarchical FL Architecture ===
🏢 Edge Servers: 2
📱 Clients per Edge: 2
📊 Total Clients: 4
🔄 Edge Rounds per Global Round: 2

🎯 Architecture Flow:
   Global Server ↔ 2 Edge Servers
   Each Edge Server ↔ 2 Clients
   Edge Servers run 2 local rounds before global aggregation

🚀 Starting Client-Edge-Global FL Simulation...


In [14]:
# Run the hierarchical federated learning simulation
# Each supernode represents an edge server that manages multiple clients internally
run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_EDGE_SERVERS,  # Edge servers as supernodes
    backend_config=backend_config,
)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=3, no round_timeout
INFO :      



🌐 === Global Server Initialization ===


INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO :      initial parameters (loss, other metrics): 0.07214174342155456, {'accuracy': 0.1}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


🌐 [Global Server] Round 0 - Global Test: loss 0.0721, accuracy 0.1000


(pid=17334) 2025-05-27 14:51:11.753069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=17334) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=17334) E0000 00:00:1748357472.025700   17334 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=17334) E0000 00:00:1748357472.054921   17334 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(ClientAppActor pid=17334) /usr/local/lib/python3.11/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=17334) given by the platformdirs library.  To remove this warning and
(ClientAppActor pid=17334) see the 

...

INFO :      aggregate_fit: received 2 results and 0 failures



🌐 [Global Server] Round 3: Aggregating 2 edge servers
🏢 Edge Server 0: 20000 examples from 2 clients (2 edge rounds)
🏢 Edge Server 1: 20000 examples from 2 clients (2 edge rounds)
(ClientAppActor pid=17334)     Epoch 2: loss 0.0331, accuracy 0.6170
(ClientAppActor pid=17334) [EdgeServer 1] Completed edge round 2
(ClientAppActor pid=17334) [EdgeServer 1] Completed all edge rounds, sending to global server
🌐 [Global Server] Aggregated 2 edge servers
📊 Total examples: 40000


INFO :      fit progress: (3, 0.03656037617921829, {'accuracy': 0.5827}, 469.44491239299987)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)


🌐 [Global Server] Round 3 - Global Test: loss 0.0366, accuracy 0.5827
(ClientAppActor pid=17334) 
(ClientAppActor pid=17334) 🏢 Creating Edge Server 0
(ClientAppActor pid=17334)   📱 Creating Client 0 for Edge Server 0
(ClientAppActor pid=17334)   📱 Creating Client 1 for Edge Server 0
(ClientAppActor pid=17334) [EdgeServer 0] Evaluating edge model
(ClientAppActor pid=17334) 
(ClientAppActor pid=17334) 🏢 Creating Edge Server 1
(ClientAppActor pid=17334)   📱 Creating Client 2 for Edge Server 1
(ClientAppActor pid=17334)   📱 Creating Client 3 for Edge Server 1
(ClientAppActor pid=17334) [EdgeServer 1] Evaluating edge model


INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 3 round(s) in 480.78s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.046551330369710923
INFO :      		round 2: 0.03960153959989548
INFO :      		round 3: 0.03694470154047012
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.07214174342155456
INFO :      		round 1: 0.04608696926236153
INFO :      		round 2: 0.03914911552071571
INFO :      		round 3: 0.03656037617921829
INFO :      	History (metrics, distributed, fit):
INFO :      	{'edge_server_ids': [(1, [0, 1]), (2, [1, 0]), (3, [0, 1])],
INFO :      	 'participating_edge_servers': [(1, 2), (2, 2), (3, 2)],
INFO :      	 'total_examples': [(1, 40000), (2, 40000), (3, 40000)]}
INFO :      	History (metrics, centralized):
INFO :      	{'accuracy': [(0, 0.1), (1, 0.4669), (2, 0.5466), (3, 0.5827)]}
INFO :      
